In [1]:
import pickle
# SKLEARN SVM PORTION

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF, DotProduct, Matern
import numpy as np
from IPython.display import clear_output, display, HTML, Javascript

# Data Acquisition

In [2]:
path = 'validation_df_072523.pkl'
X = pickle.load(open(path,'rb'))

In [7]:
# X = X[0:10]
# y = X[0:10]['label']
y =  X[:]['label']

In [8]:
# add median time column
X['median_time'] = 0 # Initialize all median_time column to 0 for all rows 

ids = np.unique(X['vid_id']) # Returns all video_id's of the dataset - sorted and without repetitions

for id in ids:  # loops through all video_id's
    vid_match_idx = X['vid_id'] == id  # boolean expression: T or F 0 - returns all indexes of entries in data with matching id
    times = X.loc[vid_match_idx]['time'] # gets the times of all data samples with matching video ID
    X.loc[vid_match_idx, 'median_time'] = np.median(times) # Now we take the median of all the times and for all videos with matching video id, 
                                                        #the median_time column gets populated with the actual median of the time stamp


# make new training and test set from X
train_idx = (X['time'] <= X['median_time'])  # Here, if the current time for each sample is less than its median_time, then the sample index is put into train_idx
test_idx = (X['time'] > X['median_time'])    # Otherwise, if the sample's time is larger than the median_time, then it goes into the test_idx

X_train = X.loc[train_idx]  #Here we build the train dataset using the train indexes from the previous section
y_train = y.loc[train_idx] # Get the matching y labels and put them into the train
X_test = X.loc[test_idx]  # Get samples with matching test idx and put it into a test data set
y_test = y.loc[test_idx] # Get y labels with corresponding indexes that match the test_idx

In [10]:
y_test

40           person
41       not_person
42           person
43       not_person
44           person
            ...    
57555    not_person
57556    not_person
57557        person
57558    not_person
57559    not_person
Name: label, Length: 27932, dtype: object

In [11]:
#TRAIN DATA

train_imgpath = X_train['img_path']
train_originallabel = X_train['original_label']
train_metadata = X_train['meta_data']
train_bboxs = X_train['bboxs']
X_train = X_train.drop(['label','original_label', 'img_path','meta_data','bboxs','median_time'], axis=1)


#VALIDATION DATA
val_imgpath = X_test['img_path']
val_originallabel = X_test['original_label']
val_metadata = X_test['meta_data']
val_bboxs = X_test['bboxs']
X_test = X_test.drop(['label','original_label', 'img_path','meta_data','bboxs', 'median_time'], axis=1)


In [12]:
data = X_test.copy()
data['label'] = y_test
data['original_label'] = val_originallabel
data['img_path'] = val_imgpath
data['metadata'] = val_metadata
data['bboxs'] = val_bboxs

In [14]:
y_train

0            person
1        not_person
2            person
3        not_person
4            person
            ...    
57510    not_person
57511    not_person
57512        person
57513    not_person
57514    not_person
Name: label, Length: 29628, dtype: object

# Base Kernels

In [15]:
#### ---------- BASE FUNCTIONS -------------

kDP_m1_feat = DotProduct(sigma_0 = 0.4)

def m1(X, Y):
    '''
    m1: 
    k = 0.8 * kDP_feat(sigma_0 = 0.4)
    '''
    X1 = X[:,0:256]
    Y1 = Y[:,0:256]
    gram_matrix = 0.8 * kDP_m1_feat(X1,Y1)
    return gram_matrix


'''
m2:
k = 2 * kRBF_feat(lengthscale = 10)
'''
kRBF_m2_feat = RBF(length_scale = 10)

def m2(X,Y):
    X1 = X[:,0:256]
    Y1 = Y[:,0:256]
    gram_matrix = 2 * kRBF_m2_feat(X1,Y1)
    return gram_matrix


## M1

In [16]:
svm1 = make_pipeline(StandardScaler(), SVC(kernel=m1, C = 0.08))
svm1.fit(X_train, y_train)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done fitting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [17]:
y_pred_svm1 = svm1.predict(X_test)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done predicting.";
  window.speechSynthesis.speak(msg);
"""))


<IPython.core.display.Javascript object>

In [18]:
data['m1'] = y_pred_svm1
data

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x254,x255,time,vid_id,label,original_label,img_path,metadata,bboxs,m1
40,1.279082,2.163162,1.387326,1.021025,2.144935,1.011914,2.294550,2.627403,4.511600,1.920112,...,2.495432,1.003313,20,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[224, 2, 640, 2, 640, 480, 2...","(224.0, 2.0, 640.0, 480.0)",person
41,0.299634,0.173886,0.229507,0.412398,1.746415,0.655757,0.477212,0.823943,1.453786,1.709671,...,0.423107,-0.898676,20,0,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[233, 203, 269, 203, 269, 44...","(233.0, 203.0, 269.0, 446.0)",person
42,1.281126,1.919473,0.949052,0.948605,1.785091,1.158464,2.384823,2.453316,4.210309,1.674176,...,2.939399,1.698413,21,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[205, 2, 640, 2, 640, 480, 2...","(205.0, 2.0, 640.0, 480.0)",person
43,0.239554,0.143336,0.363114,0.386323,1.561549,0.498190,0.261052,0.869688,1.246313,1.525887,...,0.214069,-0.805620,21,0,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[213, 189, 259, 189, 259, 42...","(213.0, 189.0, 259.0, 429.0)",person
44,0.906112,2.327980,1.081830,0.876156,1.779853,0.932696,2.281990,2.790590,4.259732,1.661546,...,2.584700,1.444308,22,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[226, 2, 640, 2, 640, 480, 2...","(226.0, 2.0, 640.0, 480.0)",person
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57555,0.157303,-0.367324,0.193963,0.355739,0.727778,0.132058,-0.090498,0.475374,0.313263,0.659360,...,-0.277401,-0.361201,29,570,not_person,sandwich,../../data/val/Charades/2KGV3/frame0726.jpg,"{'segmentation': [[566, 177, 597, 177, 597, 21...","(566.0, 177.0, 597.0, 215.0)",not_person
57556,1.623008,-0.078743,0.151457,0.437605,1.898401,1.512192,1.535586,0.100098,1.486610,1.573581,...,1.258003,1.291426,29,570,not_person,book,../../data/val/Charades/2KGV3/frame0726.jpg,"{'segmentation': [[842, 542, 1018, 542, 1018, ...","(842.0, 542.0, 1018.0, 664.0)",not_person
57557,1.040004,2.941334,1.731502,2.552450,2.376274,1.963729,3.129268,2.360919,3.387702,2.079999,...,3.175847,2.274000,30,570,person,person,../../data/val/Charades/2KGV3/frame0751.jpg,"{'segmentation': [[380, 63, 786, 63, 786, 720,...","(380.0, 63.0, 786.0, 720.0)",person
57558,0.149564,-0.377917,-0.075668,0.497504,1.054012,0.151234,0.381635,0.781065,0.611721,1.294011,...,0.039735,-0.763045,30,570,not_person,sandwich,../../data/val/Charades/2KGV3/frame0751.jpg,"{'segmentation': [[395, 367, 434, 367, 434, 40...","(395.0, 367.0, 434.0, 401.0)",not_person


## M2

In [19]:
svm2 = make_pipeline(StandardScaler(), SVC(kernel=m2, C = 0.2))
svm2.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(C=0.2, kernel=<function m2 at 0x000001CC6ED5CAF0>))])

In [20]:
y_pred_svm2 = svm2.predict(X_test)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done fitting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [21]:
data['m2'] = y_pred_svm2
data.head(5)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x255,time,vid_id,label,original_label,img_path,metadata,bboxs,m1,m2
40,1.279082,2.163162,1.387326,1.021025,2.144935,1.011914,2.294550,2.627403,4.511600,1.920112,...,1.003313,20,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[224, 2, 640, 2, 640, 480, 2...","(224.0, 2.0, 640.0, 480.0)",person,person
41,0.299634,0.173886,0.229507,0.412398,1.746415,0.655757,0.477212,0.823943,1.453786,1.709671,...,-0.898676,20,0,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[233, 203, 269, 203, 269, 44...","(233.0, 203.0, 269.0, 446.0)",person,person
42,1.281126,1.919473,0.949052,0.948605,1.785091,1.158464,2.384823,2.453316,4.210309,1.674176,...,1.698413,21,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[205, 2, 640, 2, 640, 480, 2...","(205.0, 2.0, 640.0, 480.0)",person,person
43,0.239554,0.143336,0.363114,0.386323,1.561549,0.498190,0.261052,0.869688,1.246313,1.525887,...,-0.805620,21,0,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[213, 189, 259, 189, 259, 42...","(213.0, 189.0, 259.0, 429.0)",person,person
44,0.906112,2.327980,1.081830,0.876156,1.779853,0.932696,2.281990,2.790590,4.259732,1.661546,...,1.444308,22,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[226, 2, 640, 2, 640, 480, 2...","(226.0, 2.0, 640.0, 480.0)",person,person


## Temporal Kernels - Official 

In [22]:
#### ---------- BASE FUNCTIONS -------------


'''
m3: 
C = 0.16
k = 0.5 * kDP_feat(sigma_0 = 0.9) + 3 * kRBF_time(lengthscale = 15) * kRBF_vidid(lengthscale = 1e-7)
'''

kDP_m3_feat = DotProduct(sigma_0 = 0.9)
kRBF_m3_time = RBF(length_scale = 15)
kRBF_m3_vidid = RBF(length_scale = 1e-7)

def m3(X, Y):

    X1 = X[:,0:256]
    Y1 = Y[:,0:256]
    Xt = np.reshape(X[:,256],(-1,1))
    Yt = np.reshape(Y[:,256],(-1,1))
    Xv = np.reshape(X[:,257],(-1,1))
    Yv = np.reshape(Y[:,257],(-1,1))
    gram_matrix = 0.5 * kDP_m3_feat(X1,Y1) + 3 * kRBF_m3_time(Xt,Yt) * kRBF_m3_vidid(Xv,Yv)
    return gram_matrix


'''
m4: 
C = 0.2
k = 3 * kRBF_feat(lengthscale = 10) + 1 * kRBF_time(lengthscale = 10) * kRBF_vidid(lengthscale = 1e-7)
'''

kRBF_m4_feat = RBF(length_scale = 10)
kRBF_m4_time = RBF(length_scale = 10)
kRBF_m4_vidid = RBF(length_scale = 1e-7)

def m4(X, Y):

    X1 = X[:,0:256]
    Y1 = Y[:,0:256]
    Xt = np.reshape(X[:,256],(-1,1))
    Yt = np.reshape(Y[:,256],(-1,1))
    Xv = np.reshape(X[:,257],(-1,1))
    Yv = np.reshape(Y[:,257],(-1,1))
    
    gram_matrix = 3 * kRBF_m4_feat(X1,Y1) + 1 * kRBF_m4_time(Xt,Yt) * kRBF_m4_vidid(Xv,Yv)
    return gram_matrix


'''
m5:
C = 0.03
k = 0.8 * kDP_feat(sigma_0 = 0.9) + 1 * kDP_feat(sigma_0 = 0.9) * kRBF_time(lengthscale = 30) * kRBF_vidid(lengthscale = 1e-7)
'''

kDP_m5_feat = DotProduct(sigma_0 = 0.9)
kRBF_m5_time = RBF(length_scale = 30)
kRBF_m5_vidid = RBF(length_scale = 1e-7)

def m5(X, Y):

    X1 = X[:,0:256]
    Y1 = Y[:,0:256]
    Xt = np.reshape(X[:,256],(-1,1))
    Yt = np.reshape(Y[:,256],(-1,1))
    Xv = np.reshape(X[:,257],(-1,1))
    Yv = np.reshape(Y[:,257],(-1,1))
    
    gram_matrix = 0.8 * kDP_m5_feat(X1,Y1) + 1 * kDP_m5_feat(X1,Y1) * kRBF_m5_time(Xt,Yt) * kRBF_m5_vidid(Xv,Yv)

    return gram_matrix

'''
m6:
C = 0.1
c1 = 1.5 * kRBF_feat(lengthscale = 2.0) + 1.0 * kDP_feat(sigma_0 = 1.0) * kRBF_time(lengthscale = 25) * kRBF_vidid(lengthscale = 1e-7)
'''

kRBF_m6_feat = RBF(length_scale = 2.0)
kDP_m6_feat = DotProduct(sigma_0 = 1.0)
kRBF_m6_time = RBF(length_scale = 25)
kRBF_m6_vidid = RBF(length_scale = 1e-7)

def m6(X, Y):

    X1 = X[:,0:256]
    Y1 = Y[:,0:256]
    Xt = np.reshape(X[:,256],(-1,1))
    Yt = np.reshape(Y[:,256],(-1,1))
    Xv = np.reshape(X[:,257],(-1,1))
    Yv = np.reshape(Y[:,257],(-1,1))
    
    gram_matrix = 1.5 * kRBF_m6_feat(X1,Y1) + 1.0 * kDP_m6_feat(X1,Y1) * kRBF_m6_time(Xt,Yt) * kRBF_m6_vidid(Xv,Yv)

    return gram_matrix


## M3

In [23]:
'''
m3: 
C = 0.16
'''
svm3 = make_pipeline(StandardScaler(), SVC(kernel=m3, C = 0.16))
svm3.fit(X_train, y_train)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done fitting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [24]:
y_pred_svm3 = svm3.predict(X_test)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done predicting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [25]:
data['m3'] = y_pred_svm3
data.head(5)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,time,vid_id,label,original_label,img_path,metadata,bboxs,m1,m2,m3
40,1.279082,2.163162,1.387326,1.021025,2.144935,1.011914,2.294550,2.627403,4.511600,1.920112,...,20,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[224, 2, 640, 2, 640, 480, 2...","(224.0, 2.0, 640.0, 480.0)",person,person,person
41,0.299634,0.173886,0.229507,0.412398,1.746415,0.655757,0.477212,0.823943,1.453786,1.709671,...,20,0,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[233, 203, 269, 203, 269, 44...","(233.0, 203.0, 269.0, 446.0)",person,person,not_person
42,1.281126,1.919473,0.949052,0.948605,1.785091,1.158464,2.384823,2.453316,4.210309,1.674176,...,21,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[205, 2, 640, 2, 640, 480, 2...","(205.0, 2.0, 640.0, 480.0)",person,person,person
43,0.239554,0.143336,0.363114,0.386323,1.561549,0.498190,0.261052,0.869688,1.246313,1.525887,...,21,0,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[213, 189, 259, 189, 259, 42...","(213.0, 189.0, 259.0, 429.0)",person,person,not_person
44,0.906112,2.327980,1.081830,0.876156,1.779853,0.932696,2.281990,2.790590,4.259732,1.661546,...,22,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[226, 2, 640, 2, 640, 480, 2...","(226.0, 2.0, 640.0, 480.0)",person,person,person


## M4

In [26]:
'''
m4: 
C = 0.2
'''

svm4 = make_pipeline(StandardScaler(), SVC(kernel=m4,C = 0.2))
svm4.fit(X_train, y_train)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done fitting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [27]:
y_pred_svm4 = svm4.predict(X_test)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done predicting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [28]:
data['m4'] = y_pred_svm4
data.head(5)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,vid_id,label,original_label,img_path,metadata,bboxs,m1,m2,m3,m4
40,1.279082,2.163162,1.387326,1.021025,2.144935,1.011914,2.294550,2.627403,4.511600,1.920112,...,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[224, 2, 640, 2, 640, 480, 2...","(224.0, 2.0, 640.0, 480.0)",person,person,person,person
41,0.299634,0.173886,0.229507,0.412398,1.746415,0.655757,0.477212,0.823943,1.453786,1.709671,...,0,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[233, 203, 269, 203, 269, 44...","(233.0, 203.0, 269.0, 446.0)",person,person,not_person,not_person
42,1.281126,1.919473,0.949052,0.948605,1.785091,1.158464,2.384823,2.453316,4.210309,1.674176,...,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[205, 2, 640, 2, 640, 480, 2...","(205.0, 2.0, 640.0, 480.0)",person,person,person,person
43,0.239554,0.143336,0.363114,0.386323,1.561549,0.498190,0.261052,0.869688,1.246313,1.525887,...,0,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[213, 189, 259, 189, 259, 42...","(213.0, 189.0, 259.0, 429.0)",person,person,not_person,not_person
44,0.906112,2.327980,1.081830,0.876156,1.779853,0.932696,2.281990,2.790590,4.259732,1.661546,...,0,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[226, 2, 640, 2, 640, 480, 2...","(226.0, 2.0, 640.0, 480.0)",person,person,person,person


## M5

In [29]:
svm5 = make_pipeline(StandardScaler(), SVC(kernel=m5, C = 0.03))
svm5.fit(X_train, y_train)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done fitting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [30]:
y_pred_svm5 = svm5.predict(X_test)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done predicting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [31]:
data['m5'] = y_pred_svm5
data.head(5)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,label,original_label,img_path,metadata,bboxs,m1,m2,m3,m4,m5
40,1.279082,2.163162,1.387326,1.021025,2.144935,1.011914,2.294550,2.627403,4.511600,1.920112,...,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[224, 2, 640, 2, 640, 480, 2...","(224.0, 2.0, 640.0, 480.0)",person,person,person,person,person
41,0.299634,0.173886,0.229507,0.412398,1.746415,0.655757,0.477212,0.823943,1.453786,1.709671,...,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[233, 203, 269, 203, 269, 44...","(233.0, 203.0, 269.0, 446.0)",person,person,not_person,not_person,not_person
42,1.281126,1.919473,0.949052,0.948605,1.785091,1.158464,2.384823,2.453316,4.210309,1.674176,...,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[205, 2, 640, 2, 640, 480, 2...","(205.0, 2.0, 640.0, 480.0)",person,person,person,person,person
43,0.239554,0.143336,0.363114,0.386323,1.561549,0.498190,0.261052,0.869688,1.246313,1.525887,...,not_person,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[213, 189, 259, 189, 259, 42...","(213.0, 189.0, 259.0, 429.0)",person,person,not_person,not_person,not_person
44,0.906112,2.327980,1.081830,0.876156,1.779853,0.932696,2.281990,2.790590,4.259732,1.661546,...,person,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[226, 2, 640, 2, 640, 480, 2...","(226.0, 2.0, 640.0, 480.0)",person,person,person,person,person


## M6

In [32]:
svm6 = make_pipeline(StandardScaler(), SVC(kernel=m6, C = 0.1))
svm6.fit(X_train, y_train)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done fitting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [33]:
y_pred_svm6 = svm6.predict(X_test)

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Model done predicting.";
  window.speechSynthesis.speak(msg);
"""))

<IPython.core.display.Javascript object>

In [34]:
data['m6'] = y_pred_svm6
data.head(5)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,original_label,img_path,metadata,bboxs,m1,m2,m3,m4,m5,m6
40,1.279082,2.163162,1.387326,1.021025,2.144935,1.011914,2.294550,2.627403,4.511600,1.920112,...,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[224, 2, 640, 2, 640, 480, 2...","(224.0, 2.0, 640.0, 480.0)",person,person,person,person,person,person
41,0.299634,0.173886,0.229507,0.412398,1.746415,0.655757,0.477212,0.823943,1.453786,1.709671,...,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[233, 203, 269, 203, 269, 44...","(233.0, 203.0, 269.0, 446.0)",person,person,not_person,not_person,not_person,not_person
42,1.281126,1.919473,0.949052,0.948605,1.785091,1.158464,2.384823,2.453316,4.210309,1.674176,...,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[205, 2, 640, 2, 640, 480, 2...","(205.0, 2.0, 640.0, 480.0)",person,person,person,person,person,person
43,0.239554,0.143336,0.363114,0.386323,1.561549,0.498190,0.261052,0.869688,1.246313,1.525887,...,spatula,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[213, 189, 259, 189, 259, 42...","(213.0, 189.0, 259.0, 429.0)",person,person,not_person,not_person,not_person,not_person
44,0.906112,2.327980,1.081830,0.876156,1.779853,0.932696,2.281990,2.790590,4.259732,1.661546,...,person,../../data/val/YFCC100M/v_9c29a4c4e03790d16925...,"{'segmentation': [[226, 2, 640, 2, 640, 480, 2...","(226.0, 2.0, 640.0, 480.0)",person,person,person,person,person,person


# Export CSV

In [35]:
data.to_csv('ResultsData_080623.csv',index=True)